<center>
<img src="logo.png" height="900"> 
</center>


#  Описательные статистики

В этой тетрадке мы поговорим про описательные статистики и простые визуализации.

In [1]:
import numpy as np         # библиотека для матриц и математики
import pandas as pd        # библиотека для работы с табличками
from scipy import stats    # модуль для работы со статистикой

# библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')    # стиль графиков

Опция `matplotlib inline` - позволяет рисовать графики прямо в ноутбуке.

In [2]:
%matplotlib inline

# 1. Данные

In [3]:
df = pd.read_csv("data/youtube_data_short.csv", sep = "\t")

FileNotFoundError: [Errno 2] No such file or directory: 'data/youtube_data_short.csv'

In [ ]:
print(df.shape)

In [ ]:
df[105:112]

In [ ]:
df.dtypes

__Описание колонок:__

- `title` - название видео
- `commentCount` - число комментариев
- `dislikeCount` - число дизлайков
- `likeCount`  - число лайков
- `music_style` - музыкальный стиль
- `performer` - исполнитель
- `viewCount` - число просмотров
- `commentators_uniq` - число уникальных комментаторов
- `comments_obscene_cnt` - число комментариев
- `video_age` - возраст видео в днях

Возраст видео посчитан в днях относительно `2019-03-15` (момент, когда данные собирались авторами курса).

# 2. Описательные статистики

In [ ]:
np.mean(df.likeCount.values)

In [ ]:
df.likeCount.values.mean()

Подобный результат обусловлен наличием пропусков в данных

### 2.1 Максимальное и минимальное значения

Посмотрим, какое в таблице есть максимальное и минимальное количество лайков под видео.

In [ ]:
df.likeCount.max()
df['likeCount'].max()

In [ ]:
df['likeCount'].min()

Обратите внимание, что мы можем обращаться к столбцам ещё и через точку (только если имя столбца не содержит пробелов / точек / запятых / и т.д.). Давайте посмотрим кто накопил такое большое число лайков под своим клипом. 

In [ ]:
mx = df.likeCount.max()
mx

In [ ]:
df[df.likeCount == mx]

### 2.2 Меры центральной тенденции (среднего уровня)

Меры центральной тенденции — показатели, представляющие собой ответ на вопрос: «На что похожа середина данных?». Середину можно описывать с помощью разных показателей! Давайте посмотрим на них. 

**Среднее**

В случае со средним значением «серединой» будет среднее арифметическое. Среднее значение отражает типичный показатель в наборе данных. Если мы случайно выберем один из показателей, то, скорее всего, получим значение, близкое к среднему.

In [ ]:
df.likeCount.mean()

**Медиана**

Чтобы найти медиану, данные нужно расположить в порядке возрастания. Медианой будет значение, которое совпадает с серединой набора данных. Если количество значений чётное, то берётся среднее двух значений, которые «окружают» середину.

In [ ]:
df.likeCount.median()

Медиана сильно отличается от среднего это происходит из-за выбросов, характеризующихся очедь длинным хвостом распределения.

In [ ]:
df.likeCount.hist(bins=50, density=True);

Сделаем по вертикале логарифмическую шкалу, чтобы лучше разглядеть выбросы в хвосте. 

In [ ]:
df.likeCount.hist(bins=50, log=True);

__Запомните на всю жизнь:__ выбросы - зло. Среднее значение чувствительно к выбросам, а медиана нет. 

### 2.3 Меры разброса

Выше мы посмотрели на то насколько данные типичные. Кроме типичность интересно насколько в данных сильный разброс. Меры разброса отвечают на вопрос: «Как сильно мои данные могут отличаться от типичного значения?».

**Дисперсия и стандартное отклонение**

In [ ]:
df.likeCount.var(ddof=0) # дисперсия, variance

In [ ]:
df.likeCount.var(ddof=1) # несмещённый вариант

In [ ]:
df.likeCount.std(ddof=0) # стандартное отклонение, standard deviation

In [ ]:
df.likeCount.std(ddof=1) # несмещённый вариант

### 2.4 Квантили

In [ ]:
df.likeCount.quantile(0.99)

Выходит, что у $99\%$ видео из выборки меньше $128146$ лайков, и у $1\%$ - больше. Выведем самых залайканные жанры из этого $1\%$. 

In [ ]:
q = df.likeCount.quantile(0.99)

In [ ]:
x = df[df.likeCount > q].music_style
x.value_counts()

### 2.5 Хочу всё и сразу

**Метод describe** считает всё и сразу! 

In [ ]:
df.describe()

Для каждой переменной мы видим: 

* `count` - число наблюдений, которое есть без пропусков
* `mean` - среднее значение
* `std` - стандартное отклонение
* `min` -  минимум
* `max` -  максимум
* `50%` -  медиана (половина выборки больше неё, половина меньше)
* `25%` -  25% квантиль (четверть выборки меньше, 75% больше)
* `75%` -  75% квантиль

Можно построить такую же табличку только для категориальных переменных. 


In [ ]:
df.dtypes

In [ ]:
df.describe(include='object')

* `count` -  число наблюдений, которое есть без пропусков
* `unique` - число уникальных значений (категорий), которые принимает переменная
* `top` - мода для каждой категории
* `freq` - частота, с которой встречается мода

# 3. Группировка

Если бы нас попросили посчитать, какое среднее число лайков ставят под видео разных жанров, мы могли бы узнать ответ с помощью срезов. 

In [ ]:
df[df.music_style == 'rap'].likeCount.mean()

In [ ]:
df[df.music_style == 'popsa'].likeCount.mean()

Такой код надо продублировать для каждого жанра. Это не очень эффективно. Чтобы так не делать, придумали группировки. Их обычно делают с помощью метода `groupby`.

In [ ]:
df.groupby(['music_style', 'performer'])[['likeCount', 'dislikeCount'] ].agg(['mean', 'count'])

В качестве аргумента в этой переменной нужно указать по каким переменным мы хотели бы осуществить группировку применяемой функции. В данной ситуации это жанр.

Можно делать `groupby` по нескольким колонкам, получать результаты по нескольким колонкам и даже применять несколько функций за раз! 

In [ ]:
(df['commentators_uniq'] > 0).sum()

In [ ]:
df['is_comment'] = 1*(df['commentators_uniq'] > 0)  # есть ли под видео комментарии 
df['is_comment'].value_counts()

In [ ]:
df.groupby(['music_style', 'is_comment'])[['dislikeCount', 'likeCount']].agg(['count','min','max','mean'])

# 4. Apply 

Можно применить к каждому объекту ячейки одну и ту же функцию. Это можно сделать методом `apply`. Посчитаем число букв в названии каждого клипа из таблицы: 

In [ ]:
name = 'NATASHA'
len(name)

In [ ]:
df.title.apply(len)[:10]

In [ ]:
df.title.apply(len).mean() # средняя длина названия

Можно написать свою функцию и применить её к колонке. Например, вот так можно достать первую букву каждого имени: 

In [ ]:
def my_function(name):
    return name[0]

example = "Настя"

my_function(example)

In [ ]:
df.title.apply(my_function)[-10:]

Ровно то же самое можно сделать в одну строчку с помощью лямбда-функций. Это как обычная функция, но её нигде не надо объявлять. 

In [ ]:
df.title.apply(lambda name: name[0])[-10:]

Преобразования можно делать сколь угодно сложными.

# 5. Гистограммы

In [ ]:
plt.figure(figsize=(14,7))
plt.hist(df['video_age'], bins=100);

Можно сгладить распределение.

In [ ]:
plt.figure(figsize=(14,7))

df['video_age'].hist(bins=100, density=True)
df['video_age'].plot(kind='kde', linewidth=4)
plt.xlim(0, 4000)
plt.title("Распределение возраста  видео");

Можно построить сразу много гистограмм. 

In [ ]:
columns = ['viewCount', 'likeCount', 'dislikeCount']
df[columns].hist(figsize=(20, 8),log=True);

Можно построить гистограммы для логарифмов показателей. Обычно логарифмирование используют для того, чтобы сгладить выборосы и сделать данные более гладкими. Более того, на основе логарифма можно придумать преобразование, которое нормализует данные.

In [ ]:
df[columns].apply(lambda x: np.log(x + 1)).hist(figsize=(20, 6), bins=25);

# 6. Усытые ящики

In [ ]:
df_log = df[columns].apply(lambda x: np.log(x + 1)) # снова прологарифмируем
df_log['music_style'] = df['music_style']
df_log.head()

In [ ]:
plt.figure(figsize=(14,7))

sns.boxplot(x='music_style', y='likeCount', data=df_log)

plt.xlabel('музыкальный стиль')
plt.ylabel('логарифм числа лайков');

Можно построить похожую картинку под названием виалончель. Она отражает то же самое, что и ящики с усами, но выглядит более красиво. 

In [ ]:
plt.figure(figsize=(14,7))

sns.violinplot(x='music_style', y='likeCount', 
               data=df_log, inner="quartile")

plt.xlabel('музыкальный стиль')
plt.ylabel('логарифм числа лайков');